In [2]:
from bokeh.plotting.figure import figure
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_csv('./population.csv')


lấy thông tin best_country_to_live bằng html request đơn giản 

In [3]:
import requests 

headers = {'User-Agent': 'Mozilla/5.0'}
r = requests.get('https://www.usnews.com/news/best-countries/rankings/quality-of-life?renderer=json',headers=headers)
best_country_to_live=r.json()
best_country_to_live=[i['country'] for i in best_country_to_live['rankings']][:50]
df_train=df[df['Year']==2020]
df_train=df_train[['Country','Population','Yearly%Change','MedianAge','UrbanPop%','Migrants(net)','FertilityRate']]
df_train['BestToLive']=[0]*df_train.shape[0]
df_train=df_train.reset_index(drop=True)
for i in best_country_to_live:
    try:
        index = df_train[df_train['Country']==i].index[0]
        df_train.at[index,'BestToLive']=1
    except:
        pass


Lấy 2020 ra để phân lớp. Lựa chọn phân lớp theo 2 thuộc tính (đơn giản hóa bài toán dễ tạo nhiều mô hình)

In [4]:
X = [ [df_train.iloc[i]['UrbanPop%'], df_train.iloc[i]['Migrants(net)']] for i in range(df_train.shape[0]) ]
y= list(df_train['BestToLive'])

Phân lớp dựa vào UrbanPop% và Migrants(net)

In [5]:
fig=px.scatter(df_train, x="UrbanPop%", y="Migrants(net)",
        color="BestToLive")
fig.update_layout(
    title={
        'text': "Trực quan giữa UrbanPop% và Migrants(net)",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [6]:


import random
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs

from sklearn.metrics import log_loss

X = [ [df_train.iloc[i]['UrbanPop%'], df_train.iloc[i]['Migrants(net)']] for i in range(df_train.shape[0]) ]
y= list(df_train['BestToLive'])

model = LogisticRegression()
model.fit(X, y)

#kiểm tra
#Xnew = [[80,500 * 1000],[40,100],[50,-100]]
#Xnew=[[random.randint(20,100),10**(random.randint(0,60)/10)] for i in range(500)]
Xnew=X
ynew = model.predict_proba(Xnew)

dict_={
'UrbanPop%': [i[0] for i in Xnew],
'Migrants(net)': [i[1] for i in Xnew],
'predict':[i[1] for i in ynew]
}
ynew = model.predict(Xnew)

print('Binary cross entropy loss là : ',log_loss(y, ynew))
print('Độ chính xác của mô hình : ',sum(ynew==y)/len(y))

Binary cross entropy loss là :  6.466844936886641
Độ chính xác của mô hình :  0.8127659574468085


In [7]:
fig=px.scatter(dict_, x="UrbanPop%", y="Migrants(net)",
        color="predict")
fig.update_layout(
    title={
        'text': "Kiểm tra bài toán phân lớp bằng thuộc tính UrbanPop% và Migrants(net)",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

Phân lớp dựa vào FertilityRate và Yearly%Change (drop FertilityRate==0 )

In [8]:
fig=px.scatter(df_train[df_train['FertilityRate']>0], x="Yearly%Change", y="FertilityRate",
        color="BestToLive")
fig.update_layout(
    title={
        'text': "Trực quan giữa Yearly%Change và FertilityRate",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show()

In [9]:

import random


from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs

X = [ [df_train[df_train['FertilityRate']>0].iloc[i]['Yearly%Change'], df_train[df_train['FertilityRate']>0].iloc[i]['FertilityRate']] 
                for i in range(df_train[df_train['FertilityRate']>0].shape[0]) ]
y= list(df_train[df_train['FertilityRate']>0]['BestToLive'])

model = LogisticRegression()
model.fit(X, y)

#Kiểm tra 
Xnew=X
ynew = model.predict_proba(Xnew)

dict_={
'Yearly%Change': [i[0] for i in Xnew],
'FertilityRate': [i[1] for i in Xnew],
'predict':[i[1] for i in ynew]
}
ynew = model.predict(Xnew)
print('Binary cross entropy loss là : ',log_loss(y, ynew))
print('Độ chính xác của mô hình : ',sum(ynew==y)/len(y))


Binary cross entropy loss là :  8.763605932921974
Độ chính xác của mô hình :  0.746268656716418


In [10]:
fig=px.scatter(dict_, x="Yearly%Change", y="FertilityRate",
        color="predict")
fig.update_layout(
    title={
        'text': "Kiểm tra bài toán phân lớp bằng thuộc tính FertilityRate và Yearly%Change",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

Bài toán hồi quy. Dự đóan trong tương lai.
Khá đơn giản nên ta tạo các hàm như sau. 
( Đơn giản hóa bài toán, ta chọn 1 và 2 )

In [11]:
import numpy as np
import math 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
def linear_regression_1(x,y,x_for_pred) :
    x=np.array(x).reshape((-1, 1))
    y=np.array(y)
    x_for_pred=np.array(x_for_pred).reshape((-1, 1))
    model = LinearRegression().fit(x, y)
    y_pred = model.predict(x_for_pred)
    y_pre_pred = model.predict(x)
    pred_error=math.sqrt(mean_squared_error(y,y_pre_pred))/math.sqrt((max(y_pre_pred)-min(y_pre_pred))**2 +(max(x)-min(x))**2)
    return y_pred,pred_error
def linear_regression_2(x,y,x_for_pred) :
    x=np.array(x)
    x_=PolynomialFeatures(degree=2, include_bias=False).fit_transform(x.reshape((-1, 1)))
    y=np.array(y)
    x_for_pred=np.array(x_for_pred).reshape((-1, 1))
    x_for_pred=PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_for_pred)

    model = LinearRegression().fit(x_, y)
    y_pred = model.predict( x_for_pred)
    y_pre_pred = model.predict(x_)
    pred_error=math.sqrt(mean_squared_error(y,y_pre_pred))/math.sqrt((max(y_pre_pred)-min(y_pre_pred))**2 +(max(x)-min(x))**2)
    return y_pred,pred_error
linear_regression_2(df['Year'].unique(),df[df['Country']=='China']['Population'],[2025,2030,2035,2040,2050])

(array([1.47474014e+09, 1.49906064e+09, 1.51797463e+09, 1.53148210e+09,
        1.54227747e+09]),
 0.020869189706239927)

In [12]:
def gen_linear(df,funct,x_for_pred):
    x=df['Year'].unique()
    for i in df['Country'].unique():
        mean_pred_error=0
        pred_error=[]
        df_=df[df['Country']==i]
        dict_pred={}
        dict_pred['Year']=x_for_pred
        col_pred=['Population','YearlyChange','Migrants(net)','MedianAge','FertilityRate','UrbanPopulation']
        error_linear=0
        for col in col_pred : 
            dict_pred[col],error_linear=funct(x,df_[col],x_for_pred)
            pred_error.append(error_linear)
        dict_pred['Yearly%Change']=dict_pred['YearlyChange']/dict_pred['Population']
        dict_pred['UrbanPop%']=dict_pred['UrbanPopulation']*100/dict_pred['Population']
        dict_pred['Country']=[i]*len(x_for_pred)
        dict_pred['Continent']=list(df_['Continent'].unique())*len(x_for_pred)
        mean_pred_error=sum(pred_error)/len(pred_error)
        dict_pred['LossFunctCheck']=[mean_pred_error]*len(x_for_pred)
        dict_pred = [dict(zip(dict_pred,t)) for t in zip(*dict_pred.values())]
        for row in dict_pred :
            yield row


LD1=[i for i in gen_linear(df,linear_regression_1,[2025,2030,2035,2040,2050]) ]
LD1=pd.DataFrame(LD1)
LD2=[i for i in gen_linear(df,linear_regression_2,[2025,2030,2035,2040,2050]) ]
LD2=pd.DataFrame(LD2)

In [13]:
LD1

,Year,Population,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck
0,2025,3.778951e+07,1.015350e+06,-44993.213325,16.582543,5.409869,9.517517e+06,0.026869,25.185605,Afghanistan,Asia,1.482467
1,2030,4.028908e+07,1.091289e+06,-42795.797468,16.486920,5.210928,1.027058e+07,0.027086,25.492221,Afghanistan,Asia,1.482467
2,2035,4.278864e+07,1.167229e+06,-40598.381611,16.391296,5.011988,1.102364e+07,0.027279,25.763014,Afghanistan,Asia,1.482467
3,2040,4.528821e+07,1.243168e+06,-38400.965754,16.295673,4.813048,1.177671e+07,0.027450,26.003915,Afghanistan,Asia,1.482467
4,2050,5.028734e+07,1.395047e+06,-34006.134040,16.104425,4.415167,1.328284e+07,0.027742,26.413873,Afghanistan,Asia,1.482467
...,...,...,...,...,...,...,...,...,...,...,...,...
1170,2025,1.587134e+07,2.038575e+05,-134017.823668,18.685330,3.276979,6.055059e+06,0.012844,38.150902,Zimbabwe,Africa,0.278630
1171,2030,1.678869e+07,2.075294e+05,-145549.966245,18.885654,2.957300,6.500572e+06,0.012361,38.719944,Zimbabwe,Africa,0.278630
1172,2035,1.770604e+07,2.112013e+05,-157082.108822,19.085978,2.637620,6.946084e+06,0.011928,39.230023,Zimbabwe,Africa,0.278630
1173,2040,1.862339e+07,2.148732e+05,-168614.251398,19.286302,2.317941,7.391596e+06,0.011538,39.689850,Zimbabwe,Africa,0.278630


In [14]:
fig=px.scatter(LD1[LD1['Year']==2050], x="Country", y="LossFunctCheck",range_y=[0,5],
        color="Continent")

fig.update_xaxes(visible=False)
fig.show()

Điểm trừ của Linear bậc 1 : biên độ bị lệch do ảnh hướng từ các số liệu tăng đột biến. Ví dụ giả sử Afghanistan có dân số lươn tăng theo thời gian thì dân số năm 2025 được dự đoán sẽ nhỏ hơn dân số năm 2020

In [15]:
LD2

,Year,Population,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck
0,2025,4.467987e+07,1.152376e+06,24537.823051,18.398103,4.305322,1.139103e+07,0.025792,25.494761,Afghanistan,Asia,0.436912
1,2030,5.073961e+07,1.299116e+06,62661.154521,19.240558,3.535674,1.311211e+07,0.025604,25.841964,Afghanistan,Asia,0.436912
2,2035,5.727707e+07,1.455355e+06,105605.166009,20.208889,2.689447,1.496309e+07,0.025409,26.124051,Afghanistan,Asia,0.436912
3,2040,6.429225e+07,1.621095e+06,153369.857515,21.303095,1.766641,1.694396e+07,0.025214,26.354597,Afghanistan,Asia,0.436912
4,2050,7.975576e+07,1.981075e+06,263361.280580,23.869133,-0.308712,2.129539e+07,0.024839,26.700750,Afghanistan,Asia,0.436912
...,...,...,...,...,...,...,...,...,...,...,...,...
1170,2025,1.547988e+07,1.672271e+05,-155737.231362,19.810886,3.140785,6.271365e+06,0.010803,40.513016,Zimbabwe,Africa,0.161457
1171,2030,1.619496e+07,1.519725e+05,-178491.551093,20.592771,2.750736,6.828640e+06,0.009384,42.165210,Zimbabwe,Africa,0.161457
1172,2035,1.688291e+07,1.341783e+05,-202751.706494,21.452693,2.351245,7.400911e+06,0.007948,43.836712,Zimbabwe,Africa,0.161457
1173,2040,1.754371e+07,1.138444e+05,-228517.697563,22.390652,1.942311,7.988179e+06,0.006489,45.533010,Zimbabwe,Africa,0.161457


In [16]:
fig=px.scatter(LD2[LD2['Year']==2050], x="Country", y="LossFunctCheck",range_y=[0,5],
        color="Continent")

fig.update_xaxes(visible=False)
fig.show()

Điểm trừ của LD2 : Overfit, không phù hợp để chuẩn đoán các mốc thời gian xa hơn, tuy nhiên rất tốt để chuẩn đoán các giá trị liền kề

Ta đặt mốc ổn định của thế giới là năm 1990 (tình hình thế giới có thể được coi là ít xung độ vũ trang) để cải tiến dự đoán của leaner1

In [17]:
LD3=[i for i in gen_linear(df[df['Year']>=1990],linear_regression_1,[2025,2030,2035,2040,2050]) ]
LD3=pd.DataFrame(LD3)
LD3

,Year,Population,YearlyChange,Migrants(net),MedianAge,FertilityRate,UrbanPopulation,Yearly%Change,UrbanPop%,Country,Continent,LossFunctCheck
0,2025,4.312282e+07,1.043706e+06,-70941.417972,17.952350,4.538854,1.089753e+07,0.024203,25.270914,Afghanistan,Asia,0.438159
1,2030,4.747499e+07,1.116381e+06,-89952.464963,18.336933,4.027601,1.212830e+07,0.023515,25.546706,Afghanistan,Asia,0.438159
2,2035,5.182715e+07,1.189056e+06,-108963.511954,18.721517,3.516348,1.335906e+07,0.022943,25.776180,Afghanistan,Asia,0.438159
3,2040,5.617932e+07,1.261731e+06,-127974.558945,19.106101,3.005095,1.458982e+07,0.022459,25.970099,Afghanistan,Asia,0.438159
4,2050,6.488365e+07,1.407081e+06,-165996.652927,19.875268,1.982589,1.705135e+07,0.021686,26.279893,Afghanistan,Asia,0.438159
...,...,...,...,...,...,...,...,...,...,...,...,...
1170,2025,1.531886e+07,1.870439e+05,-148825.326463,18.942374,3.607345,6.033273e+06,0.012210,39.384611,Zimbabwe,Africa,3.104178
1171,2030,1.602414e+07,1.876839e+05,-164660.079143,19.167848,3.451781,6.451871e+06,0.011713,40.263437,Zimbabwe,Africa,3.104178
1172,2035,1.672943e+07,1.883239e+05,-180494.831822,19.393322,3.296216,6.870469e+06,0.011257,41.068164,Zimbabwe,Africa,3.104178
1173,2040,1.743471e+07,1.889639e+05,-196329.584501,19.618796,3.140651,7.289066e+06,0.010838,41.807785,Zimbabwe,Africa,3.104178


In [18]:
fig=px.scatter(LD3[LD3['Year']==2050], x="Country", y="LossFunctCheck",range_y=[0,5],
        color="Continent")
fig.update_xaxes(visible=False)
fig.show()

Tóm lại bài toán dự đoán sẽ được hoàn thành như sau 

In [19]:
LD=[i for i in gen_linear(df[df['Year']>=1990],linear_regression_1,[2025,2030,2035,2040,2050]) ]+[i for i in gen_linear(df[df['Year']>=1990],linear_regression_2,[2020,2021,2022,2023,2024]) ]
LD=pd.DataFrame(LD)
LD=LD.sort_values(by=['Country','Year'],ascending=True).reset_index(drop=True)

In [20]:

LD.to_csv('populationForecast.csv')